In [8]:
from pyvi import ViTokenizer, ViPosTagger # thư viện NLP tiếng Việt
from tqdm import tqdm
import numpy as np
import gensim # thư viện NLP

In [9]:
def get_data(file):
    x = []
    y = []
    with open(file, 'r', encoding='utf8') as f:
        for line in f:
            i=14
            while(line[i]!=' '):
                i=i+1
            # lấy nhãn và nội dung
            label=line[14:i]
            content=line[i+1:]
            # xóa ký tự đặc biệt và tách từ
            content = gensim.utils.simple_preprocess(content)
            content = ' '.join(content)
            content = ViTokenizer.tokenize(content)
            x.append(content)
            y.append(label)
            
    return x,y
train_file = 'train3.txt'
x_data, y_data = get_data(train_file)
print(y_data)

['khong_ban', 'khong_ban', 'khong_ban', 'ban_hang', 'khong_ban', 'khong_ban', 'khong_ban', 'khong_ban', 'ban_hang', 'ban_hang', 'ban_hang', 'khong_ban', 'ban_hang', 'ban_hang', 'khong_ban', 'khong_ban', 'khong_ban', 'khong_ban', 'khong_ban', 'ban_hang', 'khong_ban', 'khong_ban', 'khong_ban', 'khong_ban', 'ban_hang', 'ban_hang', 'khong_ban', 'khong_ban', 'khong_ban', 'khong_ban', 'khong_ban', 'khong_ban', 'khong_ban', 'khong_ban', 'khong_ban', 'khong_ban', 'khong_ban', 'khong_ban', 'khong_ban', 'ban_hang', 'khong_ban', 'khong_ban', 'ban_hang', 'khong_ban', 'ban_hang', 'khong_ban', 'khong_ban', 'ban_hang', 'khong_ban', 'khong_ban', 'khong_ban', 'khong_ban', 'ban_hang', 'khong_ban', 'khong_ban', 'ban_hang', 'khong_ban', 'khong_ban', 'khong_ban', 'ban_hang', 'khong_ban', 'khong_ban', 'khong_ban', 'khong_ban', 'khong_ban', 'khong_ban', 'khong_ban', 'khong_ban', 'khong_ban', 'khong_ban', 'khong_ban', 'ban_hang', 'khong_ban', 'khong_ban', 'ban_hang', 'khong_ban', 'khong_ban', 'khong_ban', 'kh

In [10]:
import pickle
# lưu dữ liệu
pickle.dump(x_data, open('data/x_data3.pkl', 'wb'))
pickle.dump(y_data, open('data/y_data3.pkl', 'wb'))

In [11]:
test_file = 'test3.txt'
x_test, y_test = get_data(test_file)
print(y_test)
pickle.dump(x_test, open('data/x_test3.pkl', 'wb'))
pickle.dump(y_test, open('data/y_test3.pkl', 'wb'))

['khong_ban', 'khong_ban', 'ban_hang', 'khong_ban', 'ban_hang', 'ban_hang', 'khong_ban', 'khong_ban', 'khong_ban', 'khong_ban', 'khong_ban', 'khong_ban', 'khong_ban', 'khong_ban', 'khong_ban', 'ban_hang', 'khong_ban', 'khong_ban', 'khong_ban', 'khong_ban', 'khong_ban', 'khong_ban', 'khong_ban', 'khong_ban', 'khong_ban', 'khong_ban', 'khong_ban', 'khong_ban', 'khong_ban', 'khong_ban', 'khong_ban', 'ban_hang', 'khong_ban', 'khong_ban', 'khong_ban', 'khong_ban', 'khong_ban', 'khong_ban', 'khong_ban', 'khong_ban', 'ban_hang', 'khong_ban', 'ban_hang', 'khong_ban', 'ban_hang', 'khong_ban', 'khong_ban', 'khong_ban', 'khong_ban', 'khong_ban', 'khong_ban', 'khong_ban', 'ban_hang', 'ban_hang', 'khong_ban', 'khong_ban', 'khong_ban', 'khong_ban', 'khong_ban', 'khong_ban', 'khong_ban', 'khong_ban', 'ban_hang', 'ban_hang', 'khong_ban', 'khong_ban', 'khong_ban', 'khong_ban', 'khong_ban', 'khong_ban', 'ban_hang', 'khong_ban', 'khong_ban', 'ban_hang', 'khong_ban', 'ban_hang', 'khong_ban', 'khong_ban', 

In [40]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import TruncatedSVD
import pickle

x_data = pickle.load(open('data/x_data3.pkl', 'rb'))
y_data = pickle.load(open('data/y_data3.pkl', 'rb'))

x_test = pickle.load(open('data/x_test3.pkl', 'rb'))
y_test = pickle.load(open('data/y_test3.pkl', 'rb'))

# trích xuất đặc trưng
my_stop_words = ['thi', 'la', 'dc', 'bi', 'gi', 'thì', 'là', 'được', 'bị', 'gì', 'a', 'à', 'á']
# phương pháp tf-idf ngram level - số lượng từ đại diện tối đa 10000
tfidf_vect_ngram = TfidfVectorizer(analyzer='word', stop_words=my_stop_words, max_features=10000, ngram_range=(2, 3))
tfidf_vect_ngram.fit(x_data)

x_data_tfidf_ngram =  tfidf_vect_ngram.transform(x_data)
x_test_tfidf_ngram =  tfidf_vect_ngram.transform(x_test)

# giảm chiều của dữ liệu
svd_ngram = TruncatedSVD(n_components=300, random_state=42)
svd_ngram.fit(x_data_tfidf_ngram)

x_data_tfidf_ngram_svd = svd_ngram.transform(x_data_tfidf_ngram)
x_test_tfidf_ngram_svd = svd_ngram.transform(x_test_tfidf_ngram)

In [26]:
from sklearn.metrics import accuracy_score, classification_report
def train_model(classifier, x_data, y_data, x_test, y_test):
    
    classifier.fit(x_data, y_data)
    
    train_pred = classifier.predict(x_data)
        #val_predictions = classifier.predict(x_val)
    test_pred = classifier.predict(x_test)
    print("Test accuracy: ", accuracy_score(test_pred, y_test))
    print(classification_report(y_test, test_pred))
    for label in test_pred:
        print(label)

In [43]:
from sklearn.svm import SVC
train_model(SVC(gamma='scale'), x_data_tfidf_ngram_svd, y_data, x_test_tfidf_ngram_svd, y_test)

# Test accuracy:  0.9561739710048325
#              precision    recall  f1-score   support
#
#   ban_hang       0.89      0.75      0.81       766
#  khong_ban       0.96      0.99      0.98      5235

Test accuracy:  0.9559
              precision    recall  f1-score   support

    ban_hang       0.90      0.76      0.82      1324
   khong_ban       0.96      0.99      0.97      8676

   micro avg       0.96      0.96      0.96     10000
   macro avg       0.93      0.87      0.90     10000
weighted avg       0.95      0.96      0.95     10000

ban_hang
khong_ban
ban_hang
khong_ban
ban_hang
ban_hang
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
ban_hang
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
ban_hang
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
ban_hang
ban_hang
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
ban_hang
khong_ban
khong_ban
khong_ban

khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
ban_hang
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
ban_hang
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
ban_hang
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
ban_hang
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
ban_hang
khong_ban
khong_ban
khong_ban
khong_ban
ban_hang
khong_ban
khong_

khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
ban_hang
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
ban_hang
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
ban_hang
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
ban_hang
khong_ban
ban_hang
khong_ban
khong_ban
khong_ban
ban_hang
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
ban_hang
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
ban_hang
khong_ban
ban_hang
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
ban_hang
khong_ban
khong_ban
khong_ban
ban_hang
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
ban_hang
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
kh

khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
ban_hang
khong_ban
ban_hang
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
ban_hang
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
ban_hang
khong_ban
ban_hang
khong_ban
khong_ban
khong_ban
khong_ban
ban_hang
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
ban_hang
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
ban_hang
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ba

khong_ban
khong_ban
ban_hang
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
ban_hang
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
ban_hang
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
ban_hang
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
ban_hang
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
ban_hang
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
ban_hang
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
ban_hang
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ba

...

ban_hang
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
ban_hang
ban_hang
khong_ban
ban_hang
khong_ban
ban_hang
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
ban_hang
ban_hang
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
ban_hang
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
ban_hang
khong_ban
khong_ban
khong_ban
ban_hang
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
ban_hang
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
ban_hang
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
ban_hang
khong_ban
khong_ban
khong_ban
ban_hang
khong_ban
ban_hang
khong_ban
khong_ban
khong_ban
khong

ban_hang
khong_ban
khong_ban
khong_ban
khong_ban
ban_hang
ban_hang
khong_ban
ban_hang
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
ban_hang
khong_ban
khong_ban
khong_ban
khong_ban
ban_hang
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
ban_hang
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
ban_hang
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
ban_hang
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban

khong_ban
ban_hang
ban_hang
khong_ban
khong_ban
khong_ban
ban_hang
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
ban_hang
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
ban_hang
khong_ban
khong_ban
ban_hang
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
ban_hang
khong_ban
ban_hang
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
ban_hang
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
ban_hang
khong_ban
ban_hang
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
ban_hang
khong_ban
khong_ban
kh

khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
ban_hang
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
ban_hang
khong_ban
khong_ban
ban_hang
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
ban_hang
khong_ban
khong_ban
khong_ban
khong_ban
ban_hang
khong_ban
khong_ban
khong_ban
ban_hang
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
ban_hang
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
ban_hang
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ba

khong_ban
khong_ban
khong_ban
ban_hang
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
ban_hang
khong_ban
ban_hang
khong_ban
khong_ban
khong_ban
khong_ban
ban_hang
khong_ban
khong_ban
ban_hang
khong_ban
ban_hang
khong_ban
khong_ban
ban_hang
khong_ban
ban_hang
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
ban_hang
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
ban_hang
khong_ban
khong_ban
khong_ban


In [33]:
from sklearn.naive_bayes import BernoulliNB
train_model(BernoulliNB(), x_data_tfidf_ngram, y_data, x_test_tfidf_ngram, y_test)
# Test accuracy:  0.9398
#              precision    recall  f1-score   support
#
#    ban_hang       0.87      0.64      0.74      1324
#   khong_ban       0.95      0.99      0.97      8676

Test accuracy:  0.9408
              precision    recall  f1-score   support

    ban_hang       0.87      0.65      0.74      1324
   khong_ban       0.95      0.99      0.97      8676

   micro avg       0.94      0.94      0.94     10000
   macro avg       0.91      0.82      0.86     10000
weighted avg       0.94      0.94      0.94     10000

khong_ban
khong_ban
ban_hang
khong_ban
ban_hang
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
ban_hang
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
ban_hang
ban_hang
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
ban_hang
khong_ban
khong_ban
khong_

khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
ban_hang
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
ban_hang
khong_ban
ban_hang
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
ban_hang
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
ban_hang
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
ban_hang
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
ban_hang
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_b

khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
ban_hang
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
ban_hang
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
ban_hang
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
ban_hang
khong_ban
ban_hang
khong_ban
khong_ban
khong_ban
ban_hang
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
ban_hang
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
ban_hang
khong_ban
khong_ban
khong_ban
ban_hang
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
ban_hang
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban


khong_ban
khong_ban
ban_hang
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
ban_hang
khong_ban
ban_hang
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
ban_hang
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
ban_hang
khong_ban
ban_hang
khong_ban
khong_ban
khong_ban
khong_ban
ban_hang
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
ban_hang
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
ban_hang
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban

khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
ban_hang
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
ban_hang
khong_ban
khong_ban
ban_hang
ban_hang
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
ban_hang
ban_hang
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
ban_hang
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_b

...

khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
ban_hang
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
ban_hang
khong_ban
khong_ban
khong_ban
ban_hang
khong_ban
ban_hang
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
ban_hang
khong_ban
ban_hang
khong_ban
khong_ban
ban_hang
khong_ban
khong_ban
khong_ban
khong_ban
ban_hang
khong_ban
khong_ban
khong_ban
khong_ban
ban_hang
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
ban_hang
khong_ban
khong_ban
khong_ban
ban_hang
ban_hang
khong_ban
khong_ban
ban_hang
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
ban_hang
ban_hang
khong_ban
khong_ban
ban_hang
khong_ban
khong_

khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
ban_hang
khong_ban
ban_hang
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
ban_hang
khong_ban
khong_ban
khong_ban
ban_hang
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
ban_hang
ban_hang
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
ban_hang
khong_ban
khong_ban
khong_ban
khong_ban
ban_hang
ban_hang
khong_ban
khong_ban
khong_ban
ban_hang
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
ban_hang
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
k

ban_hang
ban_hang
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
ban_hang
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
ban_hang
khong_ban
khong_ban
ban_hang
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
ban_hang
khong_ban
khong_ban
ban_hang
khong_ban
ban_hang
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
ban_hang
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
ban_hang
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
ban_hang
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
ban_hang
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
ban_hang
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
ban_hang
khong_ban
khong_ban
ban_hang
khong_ban
khong_ban
ban_hang
khong_

khong_ban
khong_ban
ban_hang
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
ban_hang
khong_ban
khong_ban
khong_ban
ban_hang
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
ban_hang
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
ban_hang
khong_ban
khong_ban
ban_hang
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
ban_hang
ban_hang
khong_ban
ban_hang
khong_ban
khong_ban
khong_ban
khong_ban
ban_hang
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
ban_hang
khong_ban
ban_hang
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
kh

khong_ban
khong_ban
khong_ban
khong_ban
ban_hang
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
ban_hang
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
ban_hang
khong_ban
khong_ban
khong_ban
ban_hang
khong_ban
khong_ban
khong_ban
khong_ban
ban_hang
khong_ban
khong_ban
ban_hang
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
ban_hang
khong_ban
khong_ban
khong_ban
khong_ban
ban_hang
khong_ban
khong_ban
khong_ban
ban_hang
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
ban_hang
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban


In [42]:
from sklearn.linear_model import LogisticRegression
train_model(LogisticRegression(solver='lbfgs'), x_data_tfidf_ngram, y_data, x_test_tfidf_ngram, y_test)
# Test accuracy:  0.952
#              precision    recall  f1-score   support
#
#    ban_hang       0.96      0.66      0.79      1324
#   khong_ban       0.95      1.00      0.97      8676

Test accuracy:  0.9517
              precision    recall  f1-score   support

    ban_hang       0.96      0.66      0.78      1324
   khong_ban       0.95      1.00      0.97      8676

   micro avg       0.95      0.95      0.95     10000
   macro avg       0.96      0.83      0.88     10000
weighted avg       0.95      0.95      0.95     10000

khong_ban
khong_ban
ban_hang
khong_ban
ban_hang
ban_hang
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
ban_hang
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
ban_hang
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
ban_hang
ban_hang
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
ban_hang
khong_ban
khong_ban
khong_ba

khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
ban_hang
khong_ban
ban_hang
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
ban_hang
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
ban_hang
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
ban_hang
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
ban_hang
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_

khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
ban_hang
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
ban_hang
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
ban_hang
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
ban_hang
khong_ban
ban_hang
khong_ban
khong_ban
khong_ban
ban_hang
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
ban_hang
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
ban_hang
khong_ban
khong_ban
khong_ban
ban_hang
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
ban_hang
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban


khong_ban
khong_ban
ban_hang
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
ban_hang
khong_ban
ban_hang
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
ban_hang
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
ban_hang
khong_ban
ban_hang
khong_ban
khong_ban
khong_ban
khong_ban
ban_hang
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
ban_hang
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
ban_hang
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban

ban_hang
khong_ban
khong_ban
khong_ban
khong_ban
ban_hang
khong_ban
khong_ban
ban_hang
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
ban_hang
khong_ban
khong_ban
ban_hang
khong_ban
khong_ban
khong_ban
ban_hang
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
ban_hang
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
ban_hang
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
ban_hang
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
ban_hang
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban


khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
ban_hang
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
ban_hang
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
ban_hang
khong_ban
khong_ban
ban_hang
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
ban_hang
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
ban_hang
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_

khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
ban_hang
khong_ban
khong_ban
khong_ban
khong_ban
ban_hang
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
ban_hang
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
ban_hang
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
ban_hang
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
ban_hang
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
ban_hang
khong_ban
khong_ban
ban_hang
ban_hang
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
ban_hang
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban


khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
ban_hang
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
ban_hang
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
ban_hang
khong_ban
khong_ban
khong_ban
khong_ban
ban_hang
khong_ban
ban_hang
khong_ban
khong_ban
ban_hang
khong_ban
khong_ban
khong_ban
khong_ban
ban_hang
khong_ban
khong_ban
khong_ban
khong_ban
ban_hang
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
ban_hang
khong_ban
khong_ban
khong_ban
ban_hang
ban_hang
khong_ban
khong_ban
ban_hang
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
ba

khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
ban_hang
khong_ban
ban_hang
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
ban_hang
ban_hang
khong_ban
ban_hang
khong_ban
ban_hang
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
ban_hang
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
ban_hang
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
ban_hang
khong_ban
khong_ban
khong_ban
ban_hang
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
ban_hang
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
k

khong_ban
khong_ban

In [41]:
from sklearn.naive_bayes import MultinomialNB
train_model(MultinomialNB(), x_data_tfidf_ngram, y_data, x_test_tfidf_ngram, y_test)
# Test accuracy:  0.9618
#              precision    recall  f1-score   support
#
#    ban_hang       0.95      0.75      0.84      1324
#   khong_ban       0.96      0.99      0.98      8676

Test accuracy:  0.9621
              precision    recall  f1-score   support

    ban_hang       0.95      0.76      0.84      1324
   khong_ban       0.96      0.99      0.98      8676

   micro avg       0.96      0.96      0.96     10000
   macro avg       0.96      0.88      0.91     10000
weighted avg       0.96      0.96      0.96     10000

khong_ban
khong_ban
ban_hang
khong_ban
ban_hang
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
ban_hang
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
ban_hang
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
ban_hang
ban_hang
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
ban_hang
khong_ban
khong_ban
khong_b

khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
ban_hang
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
ban_hang
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
ban_hang
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
ban_hang
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
ban_hang
khong_ban
khong_ban
khong_ban
khong_ban
ban_hang
khong_

khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
ban_hang
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
ban_hang
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
ban_hang
ban_hang
khong_ban
khong_ban
khong_ban
khong_ban
ban_hang
khong_ban
ban_hang
khong_ban
khong_ban
khong_ban
ban_hang
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
ban_hang
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
ban_hang
khong_ban
ban_hang
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
ban_hang
khong_ban
khong_ban
khong_ban
ban_hang
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
ban_hang
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
kho

khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
ban_hang
khong_ban
ban_hang
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
ban_hang
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
ban_hang
khong_ban
ban_hang
khong_ban
khong_ban
khong_ban
khong_ban
ban_hang
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
ban_hang
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
ban_hang
ban_hang
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban

khong_ban
ban_hang
khong_ban
khong_ban
ban_hang
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
ban_hang
khong_ban
khong_ban
ban_hang
khong_ban
khong_ban
ban_hang
ban_hang
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
ban_hang
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
ban_hang
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
ban_hang
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
ban_hang
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
ban_hang
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
k

...

khong_ban
khong_ban
khong_ban
ban_hang
khong_ban
ban_hang
khong_ban
khong_ban
ban_hang
khong_ban
khong_ban
ban_hang
ban_hang
khong_ban
ban_hang
khong_ban
ban_hang
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
ban_hang
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
ban_hang
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
ban_hang
khong_ban
khong_ban
khong_ban
ban_hang
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
ban_hang
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
ban_hang
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
ban_hang
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
ban_hang
khong

khong_ban
khong_ban
ban_hang
khong_ban
ban_hang
ban_hang
khong_ban
khong_ban
khong_ban
khong_ban
ban_hang
ban_hang
khong_ban
ban_hang
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
ban_hang
ban_hang
khong_ban
khong_ban
khong_ban
ban_hang
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
ban_hang
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
ban_hang
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
ban_hang
khong_ban
khong_ban
khong_ban
ban_hang
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
ban_hang
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khon

khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
ban_hang
ban_hang
khong_ban
khong_ban
khong_ban
ban_hang
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
ban_hang
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
ban_hang
khong_ban
khong_ban
ban_hang
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
ban_hang
ban_hang
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
ban_hang
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
ban_hang
khong_ban
ban_hang
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
k

khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
ban_hang
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
ban_hang
khong_ban
khong_ban
ban_hang
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
ban_hang
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
ban_hang
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
ban_hang
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
ban_hang
khong_ban
khong_ban
khong_ban
khong_ban
khong_b

khong_ban
khong_ban
khong_ban
ban_hang
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
ban_hang
khong_ban
khong_ban
khong_ban
khong_ban
ban_hang
khong_ban
khong_ban
ban_hang
khong_ban
ban_hang
khong_ban
khong_ban
ban_hang
ban_hang
ban_hang
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
ban_hang
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
ban_hang
